In [ ]:
#1.1 Implement gradient-based factorisation (1 mark)

In [1]:
import torch

In [1]:
def sgd_factorize(A, rank, num_epochs=1000, lr=0.01):
  
  U = torch.rand(A.shape[0], rank)
  V = torch.rand(A.shape[1], rank)

  for epoch in range(num_epochs):
    for i in range(A.shape[0]):
      for j in range(A.shape[1]):
       
        e = A[i, j] - torch.dot(U[i], V[j])

        # Update the user and item latent matrices using the gradient descent update rule.
        U[i] = U[i] + lr * e * V[j]
        V[j] = V[j] + lr * e * U[i]

  return U, V


In [2]:


A = torch.tensor([[4, 3, 1],  # User ratings for items
                   [1, 5, 0],
                   [5, 2, 4]])


rank = 2  
num_epochs = 1000
lr = 0.01

U, V = sgd_factorize(A, rank, num_epochs, lr)

print(" Matrix (U):")
print(U)
print(" Matrix (V):")
print(V)

 Matrix (U):
tensor([[ 1.2003,  0.9556],
        [ 2.0122, -0.3757],
        [ 0.5802,  2.1084]])
 Matrix (V):
tensor([[1.0429, 2.2334],
        [2.4851, 0.2182],
        [0.1491, 1.6693]])


In [ ]:
#1.2 Factorise and compute reconstruction error (1 mark)

In [21]:
import torch
from torch.nn.functional import mse_loss


A = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [3.3359, 0.0492, 1.8374],
                  [2.9407, 0.5301, 2.2620]])

rank = 2
num_epochs = 1000
lr = 0.01

def sgd_factorize(A, rank, num_epochs=1000, lr=0.01):
  U = torch.rand(A.shape[0], rank)
  V = torch.rand(A.shape[1], rank)

 
  for epoch in range(num_epochs):
    for i in range(A.shape[0]):
      for j in range(A.shape[1]):
        e = A[i, j] - torch.dot(U[i], V[j])
        U[i] = U[i] + lr * e * V[j]
        V[j] = V[j] + lr * e * U[i]
  return U, V


U, V = sgd_factorize(A, rank)
UV_hat = torch.mm(U, V.t()) 
loss = mse_loss(UV_hat, A, reduction='sum')
print("Matrix (U):")
print(U)
print("\nMatrix (V):")
print(V)
print("\nRECONSTRUCTED Matrix (V):")
print(UV_hat)
print("\n ORIGINAL Matrix (V):")
print(A)
print("\nReconstruction Loss:")
print(loss)


Matrix (U):
tensor([[-0.1688,  0.5531],
        [ 1.8053,  0.6097],
        [ 1.3245,  1.1926]])

Matrix (V):
tensor([[ 1.5151,  0.8581],
        [-0.2859,  0.8232],
        [ 0.7832,  0.9041]])

RECONSTRUCTED Matrix (V):
tensor([[ 0.2188,  0.5036,  0.3678],
        [ 3.2582, -0.0142,  1.9650],
        [ 3.0300,  0.6031,  2.1155]])

 ORIGINAL Matrix (V):
tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])

Reconstruction Loss:
tensor(0.1223)


In [ ]:
#2.1

In [7]:

A = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [3.3359, 0.0492, 1.8374],
                  [2.9407, 0.5301, 2.2620]])

#TRUNCATED SVD HERE
def truncated_svd(A, rank):
  
    U, Sigma, Vt = torch.svd(A)
    U_trunc = U[:, :rank]
    Sigma_trunc = torch.diag(Sigma[:rank])
    Vt_trunc = Vt[:rank, :]

    A_recon = torch.mm(U_trunc, torch.mm(Sigma_trunc, Vt_trunc))
    loss = torch.norm(A - A_recon) ** 2  # L2 norm for squared error
    return A_recon, loss
rank = 9

#TRUNCATED SVD HERE
A_recon, loss = truncated_svd(A, rank)

print("Original Matrix:")
print(A)
print("Reconstructed Matrix (A_reconstructed):")
print(A_recon)
print("\nReconstruction Loss:")
print(loss)


Original Matrix:
tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])
Reconstructed Matrix (A_reconstructed):
tensor([[ 0.2749,  0.3194, -0.5718],
        [ 3.0403, -1.3382, -1.8634],
        [ 3.2480, -0.6209, -1.7636]])

Reconstruction Loss:
tensor(33.9710)


In [ ]:
#3.1

In [8]:
from typing import Tuple

def sgd_factorize_masked(A: torch.Tensor, M: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    m, n = A.shape
    U = torch.rand(m, rank)
    V = torch.rand(n, rank)

    for epoch in range(num_epochs):
        for r in range(m):
            for c in range(n):
                if M[r, c] == 1:
                    e = A[r, c] - torch.dot(U[r], V[c])
                    U[r] += lr * e * V[c]
                    V[c] += lr * e * U[r]
    return U, V

# Missing value marked as 0
A = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [0.0, 0.0492, 1.8374],  
                  [2.9407, 0.0, 2.2620]]) 
M = torch.tensor([[1, 1, 1],
                  [0, 1, 1],  
                  [1, 0, 1]])  

rank = 2

U, V = sgd_factorize_masked(A, M, rank)
# Completed MATRIX
A_completed = torch.mm(U, V.t())
print("Completed Matrix:")
print(A_completed)
    


Completed Matrix:
tensor([[ 0.3323,  0.5953,  0.1793],
        [ 2.4258,  0.0511,  1.8372],
        [ 2.9414, -0.2101,  2.2612]])


In [34]:
mse = torch.mean((A - A_completed) ** 2)
print("Mean Squared Error (MSE) between A and completed matrix:", mse.item())


Mean Squared Error (MSE) between A and completed matrix: 0.6253363490104675


In [60]:
#4.1

In [38]:
import torch
import pandas as pd

ratings_tensor = torch.load('/Users/abhashshrestha/Downloads/ratings.pt')
titles_df = pd.read_csv('/Users/abhashshrestha/Downloads/titles.csv')


beautiful_mind_row_index = titles_df[titles_df['name'] == 'A Beautiful Mind'].index.item()


def gd_factorise_masked(A: torch.Tensor, M: torch.Tensor, rank: int, num_epochs: int = 1000, lr: float = 1e-5) -> torch.Tensor:
    U = torch.rand(A.shape[0], rank)
    V = torch.rand(A.shape[1], rank)
    for e in range(num_epochs):
        err = (A - U @ V.t()) * M
        U += lr * err @ V
        V += lr * err.t() @ U
    return U, V


U, V = gd_factorise_masked(ratings_tensor, (ratings_tensor != 0).float(), rank=5)

predicted_rating = U[beautiful_mind_row_index] @ V[4].t()

#squared errors
completed_matrix = U @ V.t()
masked_ratings = ratings_tensor * (ratings_tensor != 0).float()
squared_errors = (completed_matrix - masked_ratings) ** 2
sum_squared_errors = squared_errors.sum()

print("Predicted rating for 'A Beautiful Mind':", predicted_rating.item())
print("Sum of squared errors over valid values:", sum_squared_errors.item())


Predicted rating for 'A Beautiful Mind': 3.4865875244140625
Sum of squared errors over valid values: 66435584.0


In [52]:
#Calculating error for 5th user's rating prediction for A beautiful Mind
actual_rating = ratings_tensor[beautiful_mind_row_index, 4]
predicted_rating = U[beautiful_mind_row_index] @ V[4].t()
error = predicted_rating - actual_rating

print("Actual Rating for User 5 on 'A Beautiful Mind':", actual_rating.item())
print("Predicted Rating for User 5 on 'A Beautiful Mind':", predicted_rating.item())
print("Error:", error.item())

Actual Rating for User 5 on 'A Beautiful Mind': 0.0
Predicted Rating for User 5 on 'A Beautiful Mind': 3.4865875244140625
Error: 3.4865875244140625
